# STSN for Complex-valued Data & Real-valued Weights

## Prelim. Library Import

In [1]:
import numpy as np

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'				# clears Tensorflow CPU for my mac Unix terminal
os.system('cls' if os.name == 'nt' else 'clear')	# clears the terminal window screen (clc equiv. to MATLAB)

import pickle

from weightGen import *
from layers import transmitComplex
from costMask import costMask

np.random.seed(7)		# seeding the random number generator to reproduce identical results
tf.set_random_seed(7)	# seed Tensorflow random numebr generator as well

/Users/emanuelazcona/py3env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Read in Data

In [2]:
userScatter = str( input("Scatter-points: ") )
userTime = str( input("Time units: ") )

layers = int(userTime)	# number of scatter/prop. layers to navigate through
expectedScatter = int(userScatter)

if len( userScatter ) is 1:
	userScatter = "0" + userScatter
if len( userTime ) is 1:
	userTime = "0" + userTime

fileName = 'non_zero_data/scatter' + userScatter + '_T' + userTime + '_all_'
X = np.transpose( np.genfromtxt(fileName + 'in.csv', delimiter = ',') )
Y = np.transpose( np.genfromtxt(fileName + 'out.csv', delimiter = ',') )

# X = np.ones((15,30)) # FOR TESTING
# Y = np.random.random((15,30)) # FOR TESTING

Scatter-points: 01
Time units: 10


## Print User Input Info. & Extract Number of Samples/Features

In [3]:
# Print Information
print("This model contains:")
print("\t- " + userTime + " time units")
print("\t- " + userScatter + " expected scatter points\n")

sampN, featN = X.shape	# sampN: number of training samples, featN: features per sample 

This model contains:
	- 10 time units
	- 01 expected scatter points



## Random Weight Generation For Material

In [4]:
wN = featN//4	# number of transmission weights
start = int(input("Starting Weight Index of Mask: "))	# starting index (1 <--> wN)
end = int(input("Ending Weight Index of Mask: "))		# ending index (start <--> wN)

print("\t- " + str(wN) + " total weights")
print("\t- " + str(end-start+1) + " trainable weights out of total weights (masked region)\n")

# extract arrays for trainable & frozen weights
W_left, W_train, W_right = weightCreation(start, end, wN)
W_tens = weightConcat(W_left, W_train, W_right)

Starting Weight Index of Mask: 50
Ending Weight Index of Mask: 70
	- 100 total weights
	- 21 trainable weights out of total weights (masked region)



## Tensorflow Untrainable Variable Instantiation

In [5]:
X_tens = tf.placeholder(dtype = tf.float64, shape = [sampN,featN])
Y_tens = tf.placeholder(dtype = tf.float64, shape = [sampN,featN])

## Cost Function

### Predict Function

In [6]:
Yhat_tens = transmitComplex(X_tens, W_tens, layers)		# prediction function

### Apply Zero-Mask to Output of Predict

In [7]:
Yhat_masked = costMask(Yhat_tens - Y_tens, start, end)	# masking region "we don't know" for the cost

### Least Squares Cost on Unmasked Region(s)

In [8]:
least_squares = tf.norm(Yhat_masked, ord=2)**2 / (featN - ((end*4) - (start*4-3) + 1))	#

## Optimizer Definition

In [9]:
print("Building Optimizer ... ... ...")
lr = float(input("Specify learning-rate of the model (3-decimal precision): "))
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(least_squares, var_list = [W_train])
print("Done!\n")

Building Optimizer ... ... ...
Specify learning-rate of the model (3-decimal precision): 0.08
Done!



## Training

In [ ]:
epochs = 2000
loss_tolerance = 1e-8

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
# with tf.Session() as sess:
	sess.run( tf.global_variables_initializer() )
	
	print("Tensor X:")		# show info. for X
	print(X)
	print("")

	print("Tensor W: ") 	# show info. for W total
	print(W_tens.eval())
	print("")

	# show only the trainable part of W
	print("Trainable part of W (weights " + str(start) + " through " + str(end) + "):")
	print(W_train.eval())
	print("")

	print("Tensor Y: ")		# show info. for Y
	print(Y)
	print("")

	print("--------- Starting Training ---------\n")
	for i in range(1, epochs+1):

		# run X and Y dynamically into the network per iteration
		_, loss_value = sess.run([train_op, least_squares], feed_dict = {X_tens: X, Y_tens: Y})
		
		W_tens = tf.clip_by_value(W_tens, 0.0, 1.0)	# after updating the weights clip them to stay between 0 and 1
		currStatus = [loss_value]	# status of the network for the current epoch

		# add the weights to the status of the network for current epoch
		for w in W_tens.eval()[0]:
			currStatus.append(w)

		# saves objects for every iteration
		fileFolder = "results/non_zero/n" + userScatter + "_T" + userTime + "_Mask_" + str(start) + "_" + str(end) + "_lr{0:.3f}".format(lr)

		if not os.path.exists(fileFolder):
			os.makedirs(fileFolder)
		fileName = "/epoch" + str(i) + "_lossAndWeights.p"
		pickle.dump( currStatus, open( fileFolder + fileName, "wb" ) )

		# print information for the user about loss and weights
		print("Epoch: " + str(i) + "\t\tLoss: " + str(loss_value))
		print(W_train.eval())

		if loss_value <= loss_tolerance:
			break

Tensor X:
[[ 0.1353  -0.06765  0.      ... -0.       0.      -0.     ]
 [ 0.      -0.       0.      ... -0.       0.      -0.     ]
 [ 0.      -0.       0.      ... -0.       0.      -0.     ]
 ...
 [ 0.      -0.       0.      ... -0.       0.      -0.     ]
 [ 0.      -0.       0.      ... -0.       0.      -0.     ]
 [ 0.      -0.       0.      ... -0.       0.      -0.     ]]

Tensor W: 
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1.]]

Trainable part of W (weights 50 through 70):
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]

Tensor Y: 
[[ 0.      -0.06765  0.      ... -0.       0.      -0.     ]
 [ 0.      -0.       0.      ... -0.       0.      -0.     ]
 [ 0.      -0.       0.      ... -0.       0.  